## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kuito,AO,2022-07-15 23:28:21,-12.3833,16.9333,52.70,44,0,3.40,clear sky
1,1,Butaritari,KI,2022-07-15 23:28:21,3.0707,172.7902,81.99,80,100,19.80,moderate rain
2,2,Bluff,NZ,2022-07-15 23:28:21,-46.6000,168.3333,35.80,75,82,19.28,broken clouds
3,3,Ushuaia,AR,2022-07-15 23:25:07,-54.8000,-68.3000,35.26,60,20,9.22,few clouds
4,4,Sao Filipe,CV,2022-07-15 23:28:22,14.8961,-24.4956,74.79,80,100,5.39,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,2022-07-15 23:28:21,3.0707,172.7902,81.99,80,100,19.80,moderate rain
4,4,Sao Filipe,CV,2022-07-15 23:28:22,14.8961,-24.4956,74.79,80,100,5.39,overcast clouds
5,5,Kapaa,US,2022-07-15 23:28:22,22.0752,-159.3190,84.18,67,20,13.80,few clouds
6,6,Bengkulu,ID,2022-07-15 23:28:22,-3.8004,102.2655,75.52,88,100,4.27,moderate rain
8,8,Ribeira Grande,PT,2022-07-15 23:28:23,38.5167,-28.7000,70.30,89,55,8.37,broken clouds
9,9,Hithadhoo,MV,2022-07-15 23:28:23,-0.6000,73.0833,83.61,73,78,15.75,broken clouds
10,10,San Patricio,US,2022-07-15 23:28:24,28.0170,-97.5169,87.69,64,99,11.77,overcast clouds
13,13,Tautira,PF,2022-07-15 23:28:25,-17.7333,-149.1500,74.88,73,51,24.05,light rain
15,15,Gairatganj,IN,2022-07-15 23:28:25,23.4202,78.2385,74.19,94,97,7.61,overcast clouds
17,17,Le Port,RE,2022-07-15 23:28:26,-20.9373,55.2919,67.96,68,44,5.84,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                421
City                   421
Country                418
Date                   421
Lat                    421
Lng                    421
Max Temp               421
Humidity               421
Cloudiness             421
Wind Speed             421
Current Description    421
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [10]:
clean_df.count()

City_ID                418
City                   418
Country                418
Date                   418
Lat                    418
Lng                    418
Max Temp               418
Humidity               418
Cloudiness             418
Wind Speed             418
Current Description    418
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.99,moderate rain,3.0707,172.7902,
4,Sao Filipe,CV,74.79,overcast clouds,14.8961,-24.4956,
5,Kapaa,US,84.18,few clouds,22.0752,-159.3190,
6,Bengkulu,ID,75.52,moderate rain,-3.8004,102.2655,
8,Ribeira Grande,PT,70.30,broken clouds,38.5167,-28.7000,
9,Hithadhoo,MV,83.61,broken clouds,-0.6000,73.0833,
10,San Patricio,US,87.69,overcast clouds,28.0170,-97.5169,
13,Tautira,PF,74.88,light rain,-17.7333,-149.1500,
15,Gairatganj,IN,74.19,overcast clouds,23.4202,78.2385,
17,Le Port,RE,67.96,scattered clouds,-20.9373,55.2919,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.") 

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot fin

In [15]:
# 7. Drop the rows where there is no Hotel Name.
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.99,moderate rain,3.0707,172.7902,Isles Sunset Lodge
4,Sao Filipe,CV,74.79,overcast clouds,14.8961,-24.4956,Hotel Xaguate
5,Kapaa,US,84.18,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Bengkulu,ID,75.52,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
8,Ribeira Grande,PT,70.30,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
...,...,...,...,...,...,...,...
704,Pangkalanbuun,ID,75.56,broken clouds,-2.6833,111.6167,Grand Kecubung Hotel
705,Freetown,SL,76.69,light rain,8.4840,-13.2299,The Country Lodge Hotel
707,Camacha,PT,65.62,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
708,Cairo,EG,79.56,clear sky,30.0626,31.2497,Grand Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))